In [ ]:
!pip install ultralytics -q
!pip install kaggle -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.7 MB/s eta 0:00:00


In [ ]:
import os
import shutil

if not os.path.exists('/root/.kaggle'):
    os.makedirs('/root/.kaggle')

!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

print("Kaggle API key configured.")

Kaggle API key configured.


In [ ]:
DATASET_NAME = 'sumn2u/garbage-classification-v2'
!kaggle datasets download -d {DATASET_NAME}

import zipfile
with zipfile.ZipFile('garbage-classification-v2.zip', 'r') as zip_ref:
    zip_ref.extractall('garbage-dataset')


Dataset URL: https://www.kaggle.com/datasets/sumn2u/garbage-classification-v2
License(s): MIT
 95% 745M/782M [00:03<00:00, 146MB/s] 
100% 782M/782M [00:03<00:00, 219MB/s]
Dataset downloaded and extracted.


In [ ]:
import os
import shutil
from pathlib import Path

DATA_PATH = Path('./garbage-dataset')
YOLO_ROOT = Path('YOLO_Garbage_Detection')
CLASSES = sorted([d.name for d in DATA_PATH.iterdir() if d.is_dir()])
CLASS_MAP = {name: i for i, name in enumerate(CLASSES)}

# Create YOLO structure
(YOLO_ROOT / 'images/train').mkdir(parents=True, exist_ok=True)
(YOLO_ROOT / 'labels/train').mkdir(parents=True, exist_ok=True)

print(f"Detected Classes: {CLASSES}")
print("-" * 30)

# The YOLO label is a single line: [class_id] [x_center] [y_center] [width] [height]
# For single object detection, we assume the object is 100% of the image (0.5 0.5 1.0 1.0)
YOLO_LABEL = "{} 0.5 0.5 1.0 1.0"

total_images = 0

# Process each class directory
for class_name, class_id in CLASS_MAP.items():
    class_dir = DATA_PATH / class_name
    for img_file in os.listdir(class_dir):
        if img_file.endswith(('.jpg', '.jpeg', '.png')):
            # 1. Image Path Setup
            src_img_path = class_dir / img_file
            dst_img_path = YOLO_ROOT / 'images/train' / img_file

            # 2. Label Path Setup
            label_file_name = Path(img_file).stem + '.txt'
            dst_label_path = YOLO_ROOT / 'labels/train' / label_file_name

            # 3. Copy Image
            shutil.copy(src_img_path, dst_img_path)

            # 4. Create YOLO Label File
            with open(dst_label_path, 'w') as f:
                f.write(YOLO_LABEL.format(class_id))

            total_images += 1

print(f"Successfully processed {total_images} images into YOLO format.")

Detected Classes: ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']
------------------------------
Successfully processed 20212 images into YOLO format.


In [ ]:
import yaml

data_yaml_content = {
    'path': str(YOLO_ROOT),
    'train': 'images/train',
    'val': 'images/train',
    'nc': len(CLASSES),
    'names': CLASSES
}

yaml_file_path = YOLO_ROOT / 'data.yaml'
with open(yaml_file_path, 'w') as f:
    yaml.dump(data_yaml_content, f)

print(f"data.yaml created at: {yaml_file_path}")
print(f"Classes (nc={len(CLASSES)}): {CLASSES}")

data.yaml created at: YOLO_Garbage_Detection/data.yaml
Classes (nc=10): ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'metal', 'paper', 'plastic', 'shoes', 'trash']


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

results = model.train(
    data=str(yaml_file_path),
    epochs=10,
    imgsz=640,
    batch=16,
    name='garbage_detection_yolov8n'
)

print("Training complete. Results are saved in runs/detect/garbage_detection_yolov8n")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.237 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=YOLO_Garbage_Detection/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, ker

In [ ]:
final_weights_path = 'runs/detect/garbage_detection_yolov8n/weights/best.pt'


print(f"The best trained model is saved at: {final_weights_path}")

In [ ]:
!zip -r /content/model.zip /content/runs

  adding: content/runs/ (stored 0%)
  adding: content/runs/detect/ (stored 0%)
  adding: content/runs/detect/garbage_detection_yolov8n/ (stored 0%)
  adding: content/runs/detect/garbage_detection_yolov8n/val_batch2_labels.jpg (deflated 21%)
  adding: content/runs/detect/garbage_detection_yolov8n/val_batch0_labels.jpg (deflated 20%)
  adding: content/runs/detect/garbage_detection_yolov8n/results.png (deflated 8%)
  adding: content/runs/detect/garbage_detection_yolov8n/train_batch2.jpg (deflated 15%)
  adding: content/runs/detect/garbage_detection_yolov8n/confusion_matrix.png (deflated 16%)
  adding: content/runs/detect/garbage_detection_yolov8n/results.csv (deflated 58%)
  adding: content/runs/detect/garbage_detection_yolov8n/labels.jpg (deflated 45%)
  adding: content/runs/detect/garbage_detection_yolov8n/train_batch1.jpg (deflated 21%)
  adding: content/runs/detect/garbage_detection_yolov8n/val_batch1_labels.jpg (deflated 15%)
  adding: content/runs/detect/garbage_detection_yolov8n/va